In [29]:
try:
    import re
    import numpy as np
    import pandas as pd
    from typing import List
    from pandas import DataFrame
    from tqdm import tqdm
    from time import time
    from convokit import Corpus, download
except Exception as e:
    print(e)

In [30]:
# Load the modified corpora 
corpus = Corpus("mens-rights-2018-transformed-3")

In [31]:
# Create a conversation-level dictionary 
conv_metadata_agg = {}

In [32]:
# Load the utterance dataframe 
utt_df = corpus.get_utterances_dataframe().drop("vectors", axis = 1)
# utt_df_grouped = utt_df.groupby(by = "conversation_id", as_index = True).agg({"meta.hedges_modals": ["sum"]})

In [37]:
utt_df.head()

,timestamp,text,speaker,reply_to,conversation_id,meta.lowercase_text,meta.sentiment_polarity,meta.normalized_hedge_cnt,meta.normalized_groupRef_cnt,meta.subjectivity_score,meta.politeness_markers,negative_polarity,positive_polarity,neutral_polarity,gratitude,deference,greeting,positive_lexicon,apologizing,please
id,,,,,,,,,,,,,,,,,,,,
dsdpaj9,1515441265,Nothing wrong with sex.,ee4m,7p12k7,7p12k7,Nothing wrong with sex.,"{'neg': 0.0, 'neu': 0.54, 'pos': 0.46, 'compou...",0.0,0.0,0.9,"{'feature_politeness_==Please==': 0, 'feature_...",0.0,0.46,0.54,0,0,0,0,0,0
dsdpgka,1515441436,Amount of sexual partners for women before mar...,FyrW,dsdpaj9,7p12k7,Amount of sexual partners for women before mar...,"{'neg': 0.164, 'neu': 0.682, 'pos': 0.155, 'co...",0.0,0.0,0.689394,"{'feature_politeness_==Please==': 0, 'feature_...",0.164,0.155,0.682,0,0,0,1,0,0
dsdpqf5,1515441710,"Sure, thats a non brainer. People that like se...",ee4m,dsdpgka,7p12k7,"Sure, thats a non brainer. People that like se...","{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp...",0.005263,0.0,0.732222,"{'feature_politeness_==Please==': 0, 'feature_...",0.0,0.217,0.783,0,0,0,1,0,0
dsdprlw,1515441742,A lock that opens for any key is useless.\n\nA...,ItSeemedObvious,7p12k7,7p12k7,A lock that opens for any key is useless.\n\nA...,"{'neg': 0.149, 'neu': 0.851, 'pos': 0.0, 'comp...",0.0,0.0,0.8,"{'feature_politeness_==Please==': 0, 'feature_...",0.149,0.0,0.851,0,0,0,1,0,0
dsdq5c6,1515442132,&gt; People that like sex like sex so are more...,FyrW,dsdpqf5,7p12k7,&gt; People that like sex like sex so are more...,"{'neg': 0.0, 'neu': 0.808, 'pos': 0.192, 'comp...",0.008065,0.0,0.541667,"{'feature_politeness_==Please==': 0, 'feature_...",0.0,0.192,0.808,0,0,0,1,0,0


In [35]:
%%time
utt_df["negative_polarity"] = "None"
utt_df["positive_polarity"] = "None"
utt_df["neutral_polarity"] = "None"
utt_df["gratitude"] = "None"
utt_df["deference"] = "None"
utt_df["greeting"] = "None"
utt_df["positive_lexicon"] = "None"
utt_df["apologizing"] = "None"
utt_df["please"] = "None"


for idx in tqdm(utt_df.index):
    utt_df.loc[idx, "negative_polarity"] = utt_df.loc[idx, "meta.sentiment_polarity"]["neg"]
    utt_df.loc[idx, "positive_polarity"] = utt_df.loc[idx, "meta.sentiment_polarity"]["pos"]
    utt_df.loc[idx, "neutral_polarity"]  = utt_df.loc[idx, "meta.sentiment_polarity"]["neu"]
    utt_df.loc[idx, "gratitude"]         = utt_df.loc[idx, "meta.politeness_markers"]["feature_politeness_==Gratitude=="]
    utt_df.loc[idx, "deference"]         = utt_df.loc[idx, "meta.politeness_markers"]["feature_politeness_==Deference=="]
    utt_df.loc[idx, "greeting"]          = utt_df.loc[idx, "meta.politeness_markers"]["feature_politeness_==Indirect_(greeting)=="]
    utt_df.loc[idx, "positive_lexicon"]  = utt_df.loc[idx, "meta.politeness_markers"]["feature_politeness_==HASPOSITIVE=="]
    utt_df.loc[idx, "apologizing"]       = utt_df.loc[idx, "meta.politeness_markers"]["feature_politeness_==Apologizing=="]
    utt_df.loc[idx, "please"]            = utt_df.loc[idx, "meta.politeness_markers"]["feature_politeness_==Please=="]

    # utt_df.loc[idx, "negative_polarity"] = utt_df.loc[idx"meta.sentiment_polarity"]["neg"]

100%|█████████████████████████████████████████████████████████████████████████████| 5059/5059 [00:07<00:00, 690.39it/s]

Wall time: 7.33 s


In [6]:
utt_df["meta.normalized_hedge_cnt"].value_counts()

0.000000    2643
0.008130      13
0.011905      12
0.009259      12
0.009174      12
            ... 
0.003419       1
0.003255       1
0.012212       1
0.007776       1
0.010169       1
Name: meta.normalized_hedge_cnt, Length: 1137, dtype: int64

In [174]:
utt_df["gratitude"].value_counts()

0    4946
1     113
Name: gratitude, dtype: int64

In [175]:
utt_df["positive_lexicon"].value_counts()

1    2868
0    2191
Name: positive_lexicon, dtype: int64

In [176]:
utt_df["apologizing"].value_counts()

0    4995
1      64
Name: apologizing, dtype: int64

In [11]:
utt_df["meta.politeness_markers"][0]["feature_politeness_==Please=="]

0

In [43]:
utt_df_grouped = utt_df.groupby(by = "conversation_id", as_index = True).agg({"meta.normalized_hedge_cnt": ["mean"], "meta.normalized_groupRef_cnt": ["mean"], "meta.subjectivity_score": ["mean"], "negative_polarity": ["mean"], "positive_polarity": ["mean"], "neutral_polarity": ["mean"], "gratitude": ["sum"], "deference": ["sum"], "greeting": ["sum"], "positive_lexicon": ["sum"], "apologizing": ["sum"], "please": ["sum"]})

In [48]:
utt_df_grouped = utt_df_grouped.reset_index()

In [50]:
utt_df_grouped.columns

MultiIndex([(             'conversation_id',     ''),
            (   'meta.normalized_hedge_cnt', 'mean'),
            ('meta.normalized_groupRef_cnt', 'mean'),
            (     'meta.subjectivity_score', 'mean'),
            (           'negative_polarity', 'mean'),
            (           'positive_polarity', 'mean'),
            (            'neutral_polarity', 'mean'),
            (                   'gratitude',  'sum'),
            (                   'deference',  'sum'),
            (                    'greeting',  'sum'),
            (            'positive_lexicon',  'sum'),
            (                 'apologizing',  'sum'),
            (                      'please',  'sum')],
           )

In [53]:
for idx, row in tqdm(utt_df_grouped.iterrows()):
    conv_id = str(row["conversation_id"].values[0])
    conv_metadata_agg[conv_id] = {}
    conv_metadata_agg[conv_id]["normalized_hedge_cnt"] = round(row["meta.normalized_hedge_cnt"].values[0], 3)
    conv_metadata_agg[conv_id]["normalized_groupRef_cnt"] = round(row["meta.normalized_groupRef_cnt"].values[0], 3)
    conv_metadata_agg[conv_id]["normalized_subjectivity_score"] = round(row["meta.subjectivity_score"].values[0], 3)
    conv_metadata_agg[conv_id]["normalized_negative_polarity"] = round(row["negative_polarity"].values[0], 3)
    conv_metadata_agg[conv_id]["normalized_positive_polarity"] = round(row["positive_polarity"].values[0], 3)
    conv_metadata_agg[conv_id]["normalized_neutral_polarity"] = round(row["neutral_polarity"].values[0], 3)
    conv_metadata_agg[conv_id]["gratitude"] = row["gratitude"].values[0]
    conv_metadata_agg[conv_id]["deference"] = row["deference"].values[0]
    conv_metadata_agg[conv_id]["greeting"] = row["greeting"].values[0]
    conv_metadata_agg[conv_id]["positive_lexicon"] = row["positive_lexicon"].values[0]
    conv_metadata_agg[conv_id]["apologizing"] = row["apologizing"].values[0]
    conv_metadata_agg[conv_id]["please"] = row["please"].values[0]

198it [00:00, 988.96it/s]


In [56]:
for conv_id in tqdm(corpus.get_conversation_ids()):
    convo = corpus.get_conversation(conv_id)
    convo.add_meta("hedge_cnt", conv_metadata_agg[conv_id]["normalized_hedge_cnt"])
    convo.add_meta("normalized_groupRef_cnt", conv_metadata_agg[conv_id]["normalized_groupRef_cnt"])
    convo.add_meta("normalized_subjectivity_score", conv_metadata_agg[conv_id]["normalized_subjectivity_score"])
    convo.add_meta("normalized_negative_polarity", conv_metadata_agg[conv_id]["normalized_negative_polarity"])
    convo.add_meta("normalized_positive_polarity", conv_metadata_agg[conv_id]["normalized_positive_polarity"])
    convo.add_meta("normalized_neutral_polarity", conv_metadata_agg[conv_id]["normalized_neutral_polarity"])
    convo.add_meta("gratitude", conv_metadata_agg[conv_id]["gratitude"])
    convo.add_meta("deference", conv_metadata_agg[conv_id]["deference"])
    convo.add_meta("greeting", conv_metadata_agg[conv_id]["greeting"])
    convo.add_meta("positive_lexicon", conv_metadata_agg[conv_id]["positive_lexicon"])
    convo.add_meta("apologizing", conv_metadata_agg[conv_id]["apologizing"])
    convo.add_meta("please", conv_metadata_agg[conv_id]["please"])

100%|█████████████████████████████████████████████████████████████████████████████| 198/198 [00:00<00:00, 52346.18it/s]


In [57]:
corpus.random_conversation()

Conversation({'obj_type': 'conversation', 'meta': {'hedge_cnt': 0.006, 'normalized_groupRef_cnt': 0.002, 'normalized_subjectivity_score': 0.424, 'normalized_negative_polarity': 0.155, 'normalized_positive_polarity': 0.095, 'normalized_neutral_polarity': 0.75, 'gratitude': 0, 'deference': 0, 'greeting': 1, 'positive_lexicon': 19, 'apologizing': 0, 'please': 0}, 'vectors': [], 'tree': None, 'owner': <convokit.model.corpus.Corpus object at 0x000001B58E9B1070>, 'id': '9ixsmo'})

In [58]:
corpus = "mens-rights-2018-transformed-final"    # Store final dataframe to corpus 
base_path = "C:\Sagar Study\ML and Learning\CP Sem-8\Data\Reddit\saved-corpora"
corpus.dump(CORPUS_NAME, base_path=BASE_PATH)    # Dump corpus and load 

In [ ]:
# Round off the values in the next iteration if required 

In [65]:
utt_df_grouped.head(n = 10)  # Print the first 10 rows 

,conversation_id,meta.normalized_hedge_cnt,meta.normalized_groupRef_cnt,meta.subjectivity_score,negative_polarity,positive_polarity,neutral_polarity,gratitude,deference,greeting,positive_lexicon,apologizing,please
,,mean,mean,mean,mean,mean,mean,sum,sum,sum,sum,sum,sum
0,7nqhhh,0.003259,0.002337,0.484500,0.150000,0.084733,0.765267,0,0,0,14,0,0
1,7p0kio,0.002386,0.001937,0.401424,0.094000,0.155778,0.750148,2,1,1,18,1,0
2,7p12k7,0.003929,0.002023,0.429483,0.081000,0.130108,0.788973,0,0,0,23,0,1
3,7p8w00,0.002292,0.002073,0.491247,0.095500,0.189611,0.714944,3,0,0,12,0,0
4,7pga9k,0.004131,0.001629,0.456200,0.103400,0.172250,0.724300,2,1,0,26,0,1
5,7pmu73,0.004768,0.001748,0.468359,0.100974,0.155684,0.743500,1,2,0,25,1,0
6,7psab9,0.002646,0.001941,0.463084,0.122250,0.133375,0.744375,0,0,0,9,0,0
7,7qmo76,0.005702,0.002868,0.455881,0.209765,0.086941,0.703294,0,0,0,10,1,0
8,7rxpdo,0.004365,0.001782,0.457246,0.093207,0.106724,0.800172,0,0,0,13,0,0


In [61]:
# for idx in utt_df_grouped.index:
#     print(idx)
#     utt_df_grouped.loc[idx, "meta.hedges_modals"].values[0]
#     conv_metadata_agg[str(idx)] = utt_df_grouped.loc[idx, "meta.hedges_modals"].values[0]

In [29]:
# Re-index the dataframe 
# utt_df = utt_df.reset_index()
# conv_grouped_df = utt_df.groupby(["conversation_id", "id"])
# print(utt_df_sub["conv_hedges"].nunique())